# Situation and Problem 

Some businesses tend to be quite influenced by how commodity market behaves, particularly for the ones that rely on it as a primary input, such as mills, flour factories. However, if managing it properly, we can have not only properly manage the risk, but perhaps profit from the trading a bit. To that end, how to properly manage the purchase and sell scheudule is critcal. This small project demonstrates how to leverage linear programming to optimize for the best purchase schedule, given a set of buy and sell prices for the next a little while (less than or up to a year).

## Environment Check

In [ ]:
#R version

R.Version()

$platform
[1] "x86_64-pc-linux-gnu"

$arch
[1] "x86_64"

$os
[1] "linux-gnu"

$system
[1] "x86_64, linux-gnu"

$status
[1] ""

$major
[1] "4"

$minor
[1] "2.0"

$year
[1] "2022"

$month
[1] "04"

$day
[1] "22"

$`svn rev`
[1] "82229"

$language
[1] "R"

$version.string
[1] "R version 4.2.0 (2022-04-22)"

$nickname
[1] "Vigorous Calisthenics"

In [ ]:
# set timezone

Sys.setenv(TZ="America/Toronto")

In [ ]:
# time when the notebook

Sys.Date()

[1] "2022-06-05"

In [ ]:
Sys.time()

[1] "2022-06-05 13:35:45 EDT"

To start with, we are given a series of buy and sell prices for a commodity (wheat in this case) for the next 10 months, subject to a couple of constraints:
- the warehouse can only store 20000 bushels
- in any given month, we can only buy certain amount of wheat, up to warehouse size
-  in any given month, we can only sell certain amount of wheat, up to what we have at the beginning of the month

## Install and Import Packages

In [ ]:
special_libraries <- c("lpSolveAPI", "tidyquant", "plotly")
if(!require(special_libraries)) {
  install.packages(special_libraries)
}

# Data engineering libraries
library(tidyverse)
library(magrittr)

# Data vis libraries
library(ggplot2)
library(plotly)

# Special libraries

library(lpSolveAPI)
library(tidyquant)
options("getSymbols.warning4.0"=FALSE)
options("getSymbols.yahoo.warning"=FALSE)

Loading required package: special_libraries

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘special_libraries’”
Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘globals’, ‘listenv’, ‘parallelly’, ‘gower’, ‘hardhat’, ‘ipred’, ‘furrr’, ‘warp’, ‘BH’, ‘fracdiff’, ‘lmtest’, ‘tseries’, ‘urca’, ‘RcppArmadillo’, ‘RcppRoll’, ‘future’, ‘quadprog’, ‘zoo’, ‘recipes’, ‘rsample’, ‘padr’, ‘slider’, ‘anytime’, ‘forecast’, ‘tsfeatures’, ‘Rcpp’, ‘later’, ‘PerformanceAnalytics’, ‘quantmod’, ‘lazyeval’, ‘Quandl’, ‘riingo’, ‘alphavantager’, ‘timetk’, ‘timeDate’, ‘TTR’, ‘xts’, ‘htmlwidgets’, ‘crosstalk’, ‘promises’


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0   

## Set up the context 

In this case, we use one of the most quoted wheat future contracts available in US, Kansis City (KC) Wheat Contract, Hard Red Winter Wheat

Quotes are from the most active futures exchange in North America, CME Group, retrieved from Yahoo Finance

Values are retrieved from Yahoo Finance.

The contract delivers 5 times a year has the name fashion of KEH[YY], KEK[YY], KEN[YY], KEU[YY], KEZ[YY], as March, May, Jul, Sept, Dec

Active contracts are listed for the next 15 months, so a 2-year time frame should more than suffice.

Further, here are some of the conditions that we are bringing in:
- there is a series of demands of wheat from the mill/factory will process, subject to human inputs and decisions; here we also assume that demand cannot exceed the warehouse size
- the wheat-buying amounts need to accomodate the demand, making sure that together with the beginning inventory of the month, we can satisfy the need of the period before next available futures contract available on the market
- the wheat-buying amount cannot exceed warehouse size or we don't have place to hold the bushels! 
- the amount of wheat to sell cannot exceed wthe beginning inventory during that period.
- we cannot have a ending inventory balance greater than warehouse size
- holding cost = 10% of purchase price of that period multiply by the beginning inventory of the month

## Get Real Data

In some cases, raw pricing quotes retrieved are not quite useable, therefore,we implement pricing data processing rules:
1. if all are NaN then drop the columns/months
2. if selling price = NaN, then assuming 10% off from the purchase price of the month
3. if purchase price = NaN, then assuming 10% off from the selling price of the month

In [ ]:
# sample quotes from wheat futures market 
t(getQuote(c('KEH 22.CBT','KEU22.CBT', "KEZ22.CBT"), what = yahooQF("Bid")))

,KEH 22.CBT,KEU22.CBT,KEZ22.CBT
Trade Time,NA,2022-06-03 14:19:55,2022-06-03 14:19:57
Bid,NA,1126.5,1129.5


In [ ]:
get_commodity_bid_ask_prices <- function(starting_year=year(Sys.Date()), generic_ticker_vec=c("KEH", "KEK", "KEN", "KEU", "KEZ"), delivery_month_vec=c("March", "May", "July", "September", "December"), default_bid_ask_discount=0.025, contract_size=5000){
  # create actual buy and sell times 
  year_vec <- seq(starting_year, starting_year + 2)
  actual_delivery_month_vec <- c()
    for( year in year_vec ){
      for( month in delivery_month_vec ) {
        conc_string <- paste(as.character(year),"_",month, sep = "")
        actual_delivery_month_vec <- append(actual_delivery_month_vec, conc_string)
      }
    }

  # create futures ticket symbols for quote retrieval
  futures_ticker_vec <- c()
    for( year in year_vec ){
      for (ticker in generic_ticker_vec){
        conc_string <- paste(ticker, str_sub(as.character(year), -2, -1), ".CBT", sep="")
        futures_ticker_vec <- append(futures_ticker_vec, conc_string)
      }
    }

  # create buy/sell price chart and associated mont chart. 
  commodity_price_data_raw <- getQuote(futures_ticker_vec, what = yahooQF(c("Bid", "Ask")))
  commodity_price_data_raw$actual_delivery_month <- actual_delivery_month_vec
  
  #Data processing rules are the following:
  # 1. if there is no trade time or both Bid and Ask are 0, then no market prices for the time then drop the according columns/months
  # 2. if selling price = 0, then assuming 10% off from the purchase price of the month, bid-spread ask can be changed as part of the function input
  # 3. if purchase price = 0, then assuming 10% off from the selling price of the month, bid-spread ask can be changed as part of the function input

  commodity_price_data <- commodity_price_data_raw %>%
    set_rownames(actual_delivery_month_vec) %>%
    drop_na('Trade Time') %>%
    select(Bid, Ask) %>%
    filter(!(Bid == 0 & Ask == 0)) %>%
    mutate(Bid = ifelse(Bid == 0, round(Ask*(1-default_bid_ask_discount),2), Bid)) %>%
    mutate(Ask = ifelse(Ask == 0, round(Bid*(1+default_bid_ask_discount),2), Ask)) %>%
    t()

  return_list <- list(full_timeframe = actual_delivery_month_vec, 
    full_futures_tickers = futures_ticker_vec, 
    raw_commodity_price_data = commodity_price_data_raw, 
    commodity_price_data_per_unit = commodity_price_data / contract_size, 
    selling_price_per_unit = as.numeric(as.vector(commodity_price_data["Bid",])) / contract_size,
    purchase_price_per_unit = as.numeric(as.vector(commodity_price_data["Ask",]))/ contract_size,
    timeframe_for_planning = colnames(commodity_price_data),
    number_of_decision_points = length(colnames(commodity_price_data)))
  return(return_list)
}

price_info_list <- get_commodity_bid_ask_prices()
price_info_list

,Trade Time,Bid,Ask,actual_delivery_month
,<dttm>,<dbl>,<dbl>,<chr>
KEH22.CBT,NA,NA,NA,2022_March
KEK22.CBT,NA,NA,NA,2022_May
KEN22.CBT,2022-06-03 14:19:56,1116.25,1128.0,2022_July
KEU22.CBT,2022-06-03 14:19:55,1126.50,1143.5,2022_September
KEZ22.CBT,2022-06-03 14:19:57,1129.50,1178.0,2022_December
KEH23.CBT,2022-06-03 14:19:55,1142.00,1295.0,2023_March
KEK23.CBT,2022-06-03 14:14:44,1110.00,1194.0,2023_May
KEN23.CBT,2022-06-03 14:18:16,1050.00,1110.0,2023_July
KEU23.CBT,2022-06-03 12:24:31,768.00,1125.0,2023_September


# Modelling

## Create Utility Functions 

In [ ]:
# create model variables and constraint names 
create_model_names <- function(timeframe_for_planning) {
  # create variable names based on timeframe_for_planning 
  new_varvec <- c()

  # create purchase variable names for each month on timeframe_for_planning 
  for (month in timeframe_for_planning) {
    new_varvec <- append(new_varvec, paste("wheat_to_buy_", month, sep=""))
  }

  # create selling variable names for each month on timeframe_for_planning
  for (month in timeframe_for_planning) {
    new_varvec <- append(new_varvec, paste("wheat_to_sell_", month, sep=""))
  }

  # create ending inventory variable names for each month on timeframe_for_planning
  for (month in timeframe_for_planning) {
    new_varvec <- append(new_varvec, paste("ending_inventory_", month, sep=""))
  }

  # create contraint names for each month in timeframe_for_planning
  new_constraint_vec <- c()

  # constraint 1: wheat to sell in a given period cannot exceed the beginning inventory
  for (month in timeframe_for_planning) {
    new_constraint_vec <- append(new_constraint_vec, paste("sell<=beginning_inventory_", month, sep=""))
  }

  # constraint 2: wheat to buy in a given period cannot exceed the warehouse size
  for (month in timeframe_for_planning) {
    new_constraint_vec <- append(new_constraint_vec, paste("buy<=warehousesize_", month, sep=""))
  }

  # constraint 3: wheat to buy, together with the beginning inventory, needs to satisfy demand of that period
  for (month in timeframe_for_planning) {
    new_constraint_vec <- append(new_constraint_vec, paste("buy<=begin+demand_", month, sep=""))
  }

  # constraint 4: inventory relationship
  for (month in timeframe_for_planning) {
    new_constraint_vec <- append(new_constraint_vec, paste("inventory_relationship_", month, sep=""))
  }

  # constraint 5: inventory balance must be <= warehouse size
  for (month in timeframe_for_planning) {
    new_constraint_vec <- append(new_constraint_vec, paste("ending_inventory<=warehousesize_", month, sep=""))
  }

  name_list <- list(contraint_names = new_constraint_vec, variable_names = new_varvec)
  return(name_list)
  
}

In [ ]:
nameList <- create_model_names(timeframe_for_planning = colnames(price_info_list$commodity_price_data_per_unit))
nameList
length(nameList$contraint_names)
length(nameList$variable_names)

$contraint_names
 [1] "sell<=beginning_inventory_2022_July"           
 [2] "sell<=beginning_inventory_2022_September"      
 [3] "sell<=beginning_inventory_2022_December"       
 [4] "sell<=beginning_inventory_2023_March"          
 [5] "sell<=beginning_inventory_2023_May"            
 [6] "sell<=beginning_inventory_2023_July"           
 [7] "sell<=beginning_inventory_2023_September"      
 [8] "sell<=beginning_inventory_2023_December"       
 [9] "sell<=beginning_inventory_2024_March"          
[10] "sell<=beginning_inventory_2024_July"           
[11] "buy<=warehousesize_2022_July"                  
[12] "buy<=warehousesize_2022_September"             
[13] "buy<=warehousesize_2022_December"              
[14] "buy<=warehousesize_2023_March"                 
[15] "buy<=warehousesize_2023_May"                   
[16] "buy<=warehousesize_2023_July"                  
[17] "buy<=warehousesize_2023_September"             
[18] "buy<=warehousesize_2023_December"              
[19] "buy<=warehousesize_2024_March"                 
[20] "buy<=warehousesize_2024_July"                  
[21] "buy<=begin+demand_2022_July"                   
[22] "buy<=begin+demand_2022_September"              
[23] "buy<=begin+demand_2022_December"               
[24] "buy<=begin+demand_2023_March"                  
[25] "buy<=begin+demand_2023_May"                    
[26] "buy<=begin+demand_2023_July"                   
[27] "buy<=begin+demand_2023_September"              
[28] "buy<=begin+demand_2023_December"               
[29] "buy<=begin+demand_2024_March"                  
[30] "buy<=begin+demand_2024_July"                   
[31] "inventory_relationship_2022_July"              
[32] "inventory_relationship_2022_September"         
[33] "inventory_relationship_2022_December"          
[34] "inventory_relationship_2023_March"             
[35] "inventory_relationship_2023_May"               
[36] "inventory_relationship_2023_July"              
[37] "inventory_relationship_2023_September"         
[38] "inventory_relationship_2023_December"          
[39] "inventory_relationship_2024_March"             
[40] "inventory_relationship_2024_July"              
[41] "ending_inventory<=warehousesize_2022_July"     
[42] "ending_inventory<=warehousesize_2022_September"
[43] "ending_inventory<=warehousesize_2022_December" 
[44] "ending_inventory<=warehousesize_2023_March"    
[45] "ending_inventory<=warehousesize_2023_May"      
[46] "ending_inventory<=warehousesize_2023_July"     
[47] "ending_inventory<=warehousesize_2023_September"
[48] "ending_inventory<=warehousesize_2023_December" 
[49] "ending_inventory<=warehousesize_2024_March"    
[50] "ending_inventory<=warehousesize_2024_July"     

$variable_names
 [1] "wheat_to_buy_2022_July"          "wheat_to_buy_2022_September"    
 [3] "wheat_to_buy_2022_December"      "wheat_to_buy_2023_March"        
 [5] "wheat_to_buy_2023_May"           "wheat_to_buy_2023_July"         
 [7] "wheat_to_buy_2023_September"     "wheat_to_buy_2023_December"     
 [9] "wheat_to_buy_2024_March"         "wheat_to_buy_2024_July"         
[11] "wheat_to_sell_2022_July"         "wheat_to_sell_2022_September"   
[13] "wheat_to_sell_2022_December"     "wheat_to_sell_2023_March"       
[15] "wheat_to_sell_2023_May"          "wheat_to_sell_2023_July"        
[17] "wheat_to_sell_2023_September"    "wheat_to_sell_2023_December"    
[19] "wheat_to_sell_2024_March"        "wheat_to_sell_2024_July"        
[21] "ending_inventory_2022_July"      "ending_inventory_2022_September"
[23] "ending_inventory_2022_December"  "ending_inventory_2023_March"    
[25] "ending_inventory_2023_May"       "ending_inventory_2023_July"     
[27] "ending_inventory_2023_September" "ending_inventory_2023_December" 
[29] "ending_inventory_2024_March"     "ending_inventory_2024_July"

[1] 50

[1] 30

In [ ]:
#create functions to add model constraints, right hand side and directions

create_model_constraints <- function(lp_model, number_of_decision_points, beginning_inventory, warehouse_size, demand){
  
  # add constraint group 1: wheat to sell in a given period cannot exceed the beginning inventory
  for (m in c(1:number_of_decision_points)) {
    set.row(lp_model, m, c(rep(0, times = number_of_decision_points + m-1 ), 1, rep(0, 2 * number_of_decision_points - m)))
  }
  constraint_group1_rhs <- c(beginning_inventory, rep(0, times = number_of_decision_points - 1))
  constraint_group1_dir <- rep("<=", times = number_of_decision_points)

  # add constraint group 2: wheat to buy in a given period cannot exceed the warehouse size
  for (m in c(1:number_of_decision_points)) {
    set.row(lp_model, number_of_decision_points + m, c(rep(0, times = m-1), 1, rep(0, times = 3 * number_of_decision_points - m)))
  }
  constraint_group2_rhs <- rep(warehouse_size, times = number_of_decision_points)
  constraint_group2_dir <- rep("<=", times = number_of_decision_points)

  # add constraint group 3: wheat to buy, together with the beginning inventory, needs to satisfy demand of that period
  for (m in c(1:number_of_decision_points)) {
    if (m == 1) {
      set.row(lp_model, 2 * number_of_decision_points + m,
        c(1, 
          rep(0, times = 3 * number_of_decision_points - 1) )
        )
    } else {  
      set.row(lp_model, 2 * number_of_decision_points + m, 
        c( rep( 0, times = m-1 ), 
          1, 
          rep(0, times = 2 * number_of_decision_points - 2), 
          1, 
          rep(0, number_of_decision_points - (m-1)) )
        )
    }
  }
  constraint_group3_rhs <- demand + c(-beginning_inventory, rep(0, times = number_of_decision_points - 1)) #beginning inventory balance
  constraint_group3_dir <- rep(">=", times = number_of_decision_points)

  # add constraint group 4: inventory relationship
  for (m in c(1:number_of_decision_points)) {
    # to accomodate the beginning inventory 
    if ( m == 1 ) {
      set.row(lp_model, 3 * number_of_decision_points + m,
        c(1, 
          rep(0, times = number_of_decision_points - 1), 
          -1, 
          rep(0, times = number_of_decision_points - 1),
          -1, 
          rep(0, times = number_of_decision_points - m) ) 
      )
    }  else {  
      set.row(lp_model, 3 * number_of_decision_points + m, 
        c( rep(0, times = m-1 ), 
          1, 
          rep(0, times = number_of_decision_points - 1), 
          -1, 
          rep(0, times = number_of_decision_points - 2),
          1,
          -1, 
          rep(0, times = number_of_decision_points - m) ) 
      )
    }
  }
  constraint_group4_rhs <- demand + c(-beginning_inventory, rep(0, times = number_of_decision_points - 1)) #beginning inventory balance
  constraint_group4_dir <- rep("=", times = number_of_decision_points)

  # add constraint group 5: ending inventory balance must be <= warehouse size
  for (m in c(1:number_of_decision_points)) {
    set.row(lp_model, 4 * number_of_decision_points + m, 
      c( rep( 0, times = 2 * number_of_decision_points + m-1 ), 
        1, 
        rep(0, times = number_of_decision_points - m)
        )
      )
  } 
  constraint_group5_rhs <- rep(warehouse_size, times = number_of_decision_points)
  constraint_group5_dir <- rep("<=", times = number_of_decision_points)

  # combine and add all rhs constraint values and direction to the model
  result_list <- list(
    constraint_rhs_vec=c(constraint_group1_rhs, constraint_group2_rhs, constraint_group3_rhs, constraint_group4_rhs, constraint_group5_rhs),
    constraint_dir_vec=c(constraint_group1_dir, constraint_group2_dir, constraint_group3_dir, constraint_group4_dir, constraint_group5_dir)
    )

  return(result_list)
}

In [ ]:
# create model configuration function

setup_optimization_model <- function(beginning_inventory, warehouse_size, demand, number_of_decision_points, commodity_price_table, holding_cost_as_percent_to_purchase_price=0.1, model_name="Commodity_Supply_Management_Schedule"){
  # set up the model
  lprec <- make.lp(nrow = number_of_decision_points * 5, ncol = number_of_decision_points * 3, verbose = "full")
  name.lp(lprec, model_name)

  # set up objective function
  lp.control(lprec, sense = 'max')
  set.objfn(lprec, c(-1 * as.numeric(as.vector(commodity_price_table["Ask", ])), as.numeric(as.vector(commodity_price_table["Bid", ])), -1 * holding_cost_as_percent_to_purchase_price * as.numeric(as.vector(commodity_price_table["Bid", ]))))

  # set non-negative variable bounds
  set.bounds(lprec, lower = rep(0, number_of_decision_points * 3))

  # create and assign model variables and constraint names 
  namelist <- create_model_names(timeframe_for_planning = colnames(commodity_price_table))
  dimnames(lprec) <- namelist

  # get constraints information
  constraintlist <- create_model_constraints(lp_model=lprec, number_of_decision_points = number_of_decision_points, beginning_inventory = beginning_inventory, warehouse_size=warehouse_size, demand = demand)

  # set right hand sand of the model
  set.rhs(lprec, constraintlist$constraint_rhs_vec)

  # set relationship/direction to the right hand side
  set.constr.type(lprec, constraintlist$constraint_dir_vec)

  model_outputs_list <- list(model=lprec, constraintlist=namelist$contraint_names, variablenamelist=namelist$variable_names)

  return(model_outputs_list)
}

## Create Model

In [ ]:
lp_model <- setup_optimization_model(beginning_inventory=6000, 
    warehouse_size=20000, 
    demand=c(7500, 8500, 16350, 17500, 4500, 6000, 7000, 8500, 14000, 15000), 
    number_of_decision_points=price_info_list$number_of_decision_points, 
    commodity_price_table=price_info_list$commodity_price_data_per_unit, 
    holding_cost_as_percent_to_purchase_price=0.1, 
    model_name="Commodity_Supply_Management_Schedule"
    )

## Model Outputs

In [ ]:
# due to the large the model configuration, 30 columns and 50 rows in this case, the following statement is needed to print out the full model, which is to accessible as a text file
# the following code is not essential to the rest of the programming 

write.lp(lp_model$model, filename="model_details_output.txt")

In [ ]:
lp_model

$model
Model name: Commodity_Supply_Management_Schedule
  a linear program with 30 decision variables and 50 constraints

$constraintlist
 [1] "sell<=beginning_inventory_2022_July"           
 [2] "sell<=beginning_inventory_2022_September"      
 [3] "sell<=beginning_inventory_2022_December"       
 [4] "sell<=beginning_inventory_2023_March"          
 [5] "sell<=beginning_inventory_2023_May"            
 [6] "sell<=beginning_inventory_2023_July"           
 [7] "sell<=beginning_inventory_2023_September"      
 [8] "sell<=beginning_inventory_2023_December"       
 [9] "sell<=beginning_inventory_2024_March"          
[10] "sell<=beginning_inventory_2024_July"           
[11] "buy<=warehousesize_2022_July"                  
[12] "buy<=warehousesize_2022_September"             
[13] "buy<=warehousesize_2022_December"              
[14] "buy<=warehousesize_2023_March"                 
[15] "buy<=warehousesize_2023_May"                   
[16] "buy<=warehousesize_2023_July"                 

## Model Results

In [ ]:
#view overall model results
lp_model_object <- lp_model$model
solve(lp_model_object)


Model name:  'Commodity_Supply_Management_Schedule' - run #1    
Objective:   Maximize(R0)
 
SUBMITTED
Model size:       50 constraints,      30 variables,           88 non-zeros.
Sets:                                   0 GUB,                  0 SOS.
 
 
CONSTRAINT CLASSES
General LPSREAL   50
 
Using DUAL simplex for phase 1 and PRIMAL simplex for phase 2.
The primal and dual simplex pricing strategy set to 'Devex'.
 
Objective value     -86442.5968622 at iter          5.
Objective value     -144380.093577 at iter         10.
Objective value     -180415.921194 at iter         15.
Found feasibility by dual simplex after            17 iter.
 
Primal objective:
 
  Column name                      Value   Objective         Min         Max
  --------------------------------------------------------------------------
  wheat_to_buy_2022_July         -0.2256      -338.4      -1e+30    -0.22325
  wheat_to_buy_2022_September      -0.2287    -1943.95   -0.245575     -0.2253
  wheat_to_buy_2022

[1] 0

On surface, this doesn't sound vary appealing. After all, with all of the buys and sells planned, the mill/factory only ends up with a huge net cashoutflow. Unfortunately, this might just have to be the case. 

Here is one way to look at this. 

Notice the time when the future prices are fetched, June 2022, still during Russian-Ukraine war. If anything should've caught the eye even before the linear model is configured and run, that is the big backwardation in the wheat futures prices. In a normal economic environment, the price the commodity future should follow a upward trend, or contango. With the war crisis evolving into a supply crisis, in the near term, wheat prices will be higher than the equivalents in the more distant future. Therefore, the wheat that the business has to buy will just needs to suffice the demand, together with beginning inventory here. With the holding costs considered, 0 inventory policy is further confirmed when we look at all of the ending inventories will be just 0. With a decreasing price into the future, plus the holding costs over time, there is really no point in buying extra, holding it and waiting for selling it at a higher price later. However, that said, this is abnormal environment, it doesn't mean there is no place for this model to exist anymore.  

In [ ]:
sensitivity_analysis_results <- function(lp_model){
  # to display all rows in a dataframe
  options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
  
  #view logsitic/supply assignment: i.e. what to buy and what to sell, including how much to keep at the end of each transaction month
  #sensitivity analysis results on decision variables 
  decision_variables_df <- data.frame("Decision_Variables" = lp_model$variablenamelist, 
  "units" = get.variables(lp_model$model), 
  "reduced_costs" = tail(get.sensitivity.rhs(lp_model$model)$duals, n=length(lp_model$variablenamelist)),
  "lower_limits" = get.sensitivity.objex(lp_model$model)$objfrom, 
  "upper_limits" = get.sensitivity.objex(lp_model$model)$objtill
  )

  constraints_df <- data.frame("constraints" = c(lp_model$constraintlist, lp_model$variablenamelist),
    "shadow_price" = get.sensitivity.rhs(lp_model$model)$duals,
    "lower_limits" = get.sensitivity.rhs(lp_model$model)$dualsfrom,
    "upper_limits" = get.sensitivity.rhs(lp_model$model)$dualstill
  )[1:length(lp_model$constraintlist),]
  

  results_list <- list("Decision_Variables"=decision_variables_df, "constraints" = constraints_df)
  return(results_list)
}

sensitivity_analysis_results(lp_model=lp_model)

Decision_Variables,units,reduced_costs,lower_limits,upper_limits
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
wheat_to_buy_2022_July,1500,0.0000000,-1.000000e+30,-2.232500e-01
wheat_to_buy_2022_September,8500,0.0000000,-2.455750e-01,-2.253000e-01
wheat_to_buy_2022_December,20000,0.0000000,-2.364100e-01,1.000000e+30
wheat_to_buy_2023_March,13850,0.0000000,-2.738200e-01,-2.581900e-01
wheat_to_buy_2023_May,4500,0.0000000,-2.818400e-01,-1.998000e-01
wheat_to_buy_2023_July,6000,0.0000000,-2.442000e-01,-2.100000e-01
wheat_to_buy_2023_September,7000,0.0000000,-2.430000e-01,-2.117400e-01
wheat_to_buy_2023_December,8500,0.0000000,-2.403600e-01,-2.030500e-01
wheat_to_buy_2024_March,14000,0.0000000,-2.474050e-01,-1.990960e-01


**Action Plan**

All the variables with a name starts with "wheat" have their "final value" to indicate the transactional amount of wheat (number of bushels to buy or sell) should be in place at the futures contract delivery point. 

------

**Sensitivity Analysis-Reduced Costs**

Some important insights can be further drawn from the sensitivity analysis above. 

"Reduced costs" is important when assessing how much some price factors have to change before they can be included as part of the "final values" for a different optimal outcome. 

All reduced costs for "ending_inventory" variables indicate how much the holding costs has to lower before we should start building inventory. Echoing to the backwardation phenomenon, all the reduced costs for all transaction points considered here will essentially forces the holding costs to become "holding profits", which is just not possible. Therefore, there is no point indeed to hold any inventory, use/sell if you can is the story.

All reduced costs for "wheat_to_buy" series of variables are 0, since they are part of the optimal solution and indeed we are buying the minimum just to satisfy all the demands here. Nothing needs to "better".

All reduced costs for "wheat_to_sell" series of variables, most likely these reflect the bid-ask spread between buy and sell futures prices. In other words, when the spread closes up to 0, the decision variables will start changing. 

For a month where there is inventory to end with, in that month to sell one more unit, it takes a lot more: first, it needs to address the holding cost from inventory incurred in previous month; next, there needs to be higher selling price to close up the bid-ask spread just like any other delivery month, and finally the selling price needs further to account for the opportunity cost not selling in the previous month. For most months above, again, because we are looking at a steady backwardation trend on the wheat price, most of the reduced prices simply reflects the bid-ask spread.

-------

**Sensitivity Analysis-Shadow Price**

On the constraint side, 2 highly valuable groups of insights reside with shadow prices and slack. Slacks only show up when a constraint becomes non-binding, in other words, not used up. 

Shadow prices behave the opposite; they only have non-zero values when the constraint is binding, or used up. For instance, none of the ending_inventory constraints have a shadow price, since the cap of the ending inventory is up to the warehouse size, and none of the decision variables will push the ending inventory that high. 

Essentially, what shadow price can tell how much a constraint is worth to us to the extent that we are willing to relax such constraint by 1 unit. All shadow prices are interpretable, i.e. they have specific contextual meaning, but not all necessarily all that useful for decision-making. Take a look at the "inventory_relationship" series, which is to state that if we want to have one more bushel at the end of that delivery time, we need to pay for it at the purchase price then. It is interpretable, but not that meaningful. However, some more meaningful series are "buy<=begin+demand" series, these shadow prices tell the bid-ask gap or how much the purchase has to get better before we can consider allowing to buy more. Also, it indicates that this is the time where we are maxing out on buying opportunities.